In [1]:
%matplotlib
%load_ext autoreload
%autoreload 2

Using matplotlib backend: TkAgg


In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from scipy import interpolate
from mpl_toolkits.mplot3d import Axes3D
from pandas import read_csv
from python.utils import *

In [3]:
chanloc = '/mnt/elephant_sized_space/toaugment/CC021/chanlocs.csv'
f = '/mnt/elephant_sized_space/toaugment/CC025/MEG/PDK/epoch_55.csv'
cols = ['sph_theta', 'sph_phi', 'sph_radius']
colcart = ['X', 'Y', 'Z']
grid_x, grid_y = np.mgrid[0:100, 0:100]

In [4]:
chans = read_csv(chanloc)[colcart].as_matrix()[36:187]
locs = chan2spatial(chanloc, grid=100)
x = read_csv(f).as_matrix()[np.arange(0, 13999, 20), 36:187]

In [5]:
def gen(x, locs):
    data = np.zeros([x.shape[0], 100, 100])
    for i in range(x.shape[0]):
        data[i, :, :] = interpolate.griddata(locs, x[i, :], (grid_x, grid_y), method='cubic', rescale=True, fill_value=0)
    return data

In [6]:
cov = 0.1*np.eye(2)

n1 = gen(x, locs + np.random.multivariate_normal([0, 0], cov, locs.shape[0]))
n2 = gen(x, locs + np.random.multivariate_normal([0, 0], cov, locs.shape[0]))
n3 = gen(x, locs + np.random.multivariate_normal([0, 0], cov, locs.shape[0]))
n4 = gen(x, locs + np.random.multivariate_normal([0, 0], cov, locs.shape[0]))

averaged = (n1+n2+n3+n4)/4

In [13]:
fig = plt.figure() 
ax = fig.add_subplot(111, projection='3d')

d = data[10, :, :]
d /= d.max()
ax.scatter(chans[:, 0], chans[:, 1], chans[:, 2])
surf = ax.plot_surface(grid_x, grid_y, np.zeros_like(grid_x), rstride=1, cstride=1, facecolors=plt.cm.jet(d))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """
/usr/local/lib/python3.5/dist-packages/matplotlib/colors.py:496: RuntimeWarning: invalid value encountered in less
  cbook._putmask(xa, xa < 0.0, -1)


IndexError: index 20 is out of bounds for axis 0 with size 20

In [76]:
def plotsample(t):
    d = data[t, :, :]
    d /= d.max()
    ax.plot_surface(grid_x, grid_y, np.zeros_like(grid_x), rstride=1, cstride=1, facecolors=plt.cm.jet(d))

a = anim.FuncAnimation(fig, plotsample, frames=range(d.shape[0]), interval=200)
plt.show()

In [7]:
fig, axarr = plt.subplots(2, 3)

im1 = axarr[0, 0].imshow(n1[0, :, :], animated=True, cmap=plt.cm.cubehelix)
im2 = axarr[0, 1].imshow(n2[0, :, :], animated=True, cmap=plt.cm.cubehelix)
im3 = axarr[1, 0].imshow(n3[0, :, :], animated=True, cmap=plt.cm.cubehelix)
im4 = axarr[1, 1].imshow(n4[0, :, :], animated=True, cmap=plt.cm.cubehelix)
im5 = axarr[1, 2].imshow(averaged[0, :, :], animated=True, cmap=plt.cm.cubehelix)

def plotsample(t):
    im1.set_array(n1[t, :, :])
    im2.set_array(n2[t, :, :])
    im3.set_array(n3[t, :, :])
    im4.set_array(n4[t, :, :])
    im5.set_array(averaged[t, :, :])

a = anim.FuncAnimation(fig, plotsample, frames=range(x.shape[0]), interval=5)

# plt.show()